In [2]:
#importação pacotes
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:
# criar dataframe para receber
df_hotmart = pd.DataFrame(columns=['nome_curso', 'preco', 'autor', 'avaliação', 'qtd_avaliação', 'categoria'])

#instalar driver atual e iniciar
driver = webdriver.Chrome(ChromeDriverManager().install())

# passar por cada link e fazer a extração
for categoria in ['enem', 'pr%C3%A9-vestibular', 'vestibular']:
    
    # gerar o link
    link = 'https://hotmart.com/en/marketplace/products?initialSelectedCategory=&q=' + categoria
    
    # passar para o navegador o link de acesso
    driver.get(link)
    time.sleep(5)
    for i in range(1,20):
        time.sleep(1)
        driver.find_element(By.CLASS_NAME,"load-more").click();
    for numero in range(1,457):
        # saber a div para a busca 
        divnu = str(numero+1)
        nome = driver.find_element(By.XPATH, f'/html/body/div[1]/main/div/div[2]/div/div[2]/div/div/div/div/div[1]/div[{divnu}]/a/div[2]/div[1]/div/div[1]').text
        preco = driver.find_element(By.XPATH, f'/html/body/div[1]/main/div/div[2]/div/div[2]/div/div/div/div/div[1]/div[{divnu}]/a/div[2]/div[2]/div/h3').text
        autor = driver.find_element(By.XPATH, f'/html/body/div[1]/main/div/div[2]/div/div[2]/div/div/div/div/div[1]/div[{divnu}]/a/div[2]/div[1]/div/div[2]').text
        # tentar buscar a avaliação do curso, pois alguns não possuem esse atributo
        try:
            avaliação = driver.find_element(By.XPATH, f'/html/body/div[1]/main/div/div[2]/div/div[2]/div/div/div/div/div[1]/div[{divnu}]/a/div[2]/div[1]/div/div[3]/div/span[1]').text
            qtd_avaliação = driver.find_element(By.XPATH, f'/html/body/div[1]/main/div/div[2]/div/div[2]/div/div/div/div/div[1]/div[{divnu}]/a/div[2]/div[1]/div/div[3]/div/span[2]').text
            qtd_avaliação = qtd_avaliação.replace('(', '')
            qtd_avaliação = qtd_avaliação.replace(')', '')

        except: 
            avaliação = ''
            qtd_avaliação = ''  
        df_hotmart.loc[len(df_hotmart)] = (nome,preco,autor,avaliação,qtd_avaliação,categoria)

# fechar o navegador
driver.quit()

c:\Users\victo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/main/div/div[2]/div/div[2]/div/div/div/div/div[1]/div[2]/a/div[2]/div[2]/div/h3"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0081F243]
	(No symbol) [0x007A7FD1]
	(No symbol) [0x0069D04D]
	(No symbol) [0x006CC0B0]
	(No symbol) [0x006CC22B]
	(No symbol) [0x006FE612]
	(No symbol) [0x006E85D4]
	(No symbol) [0x006FC9EB]
	(No symbol) [0x006E8386]
	(No symbol) [0x006C163C]
	(No symbol) [0x006C269D]
	GetHandleVerifier [0x00AB9A22+2655074]
	GetHandleVerifier [0x00AACA24+2601828]
	GetHandleVerifier [0x008C8C0A+619850]
	GetHandleVerifier [0x008C7830+614768]
	(No symbol) [0x007B05FC]
	(No symbol) [0x007B5968]
	(No symbol) [0x007B5A55]
	(No symbol) [0x007C051B]
	BaseThreadInitThunk [0x76A87D69+25]
	RtlInitializeExceptionChain [0x77C1BB9B+107]
	RtlClearBits [0x77C1BB1F+191]


In [ ]:
df_hotmart['categoria'].replace(['pr%C3%A9-vestibular'],'pré-vestibular', inplace=True)

In [25]:
# exportar para o excel
df_hotmart.to_excel('df_hotmart.xlsx', index=False)

In [16]:
# ler df
df_hotmart = pd.read_excel('df_hotmart.xlsx')

In [19]:
# achar linhas para limpar
lista_deletar = []
for i in df_hotmart.index:
    valor = str(df_hotmart.at[i, 'nome_curso'])
    valor = valor.lower()
    if 'enem' not in valor or 'vestibular' not in valor:
        lista_deletar.append(i)

In [20]:
# limpar linhas
for i in lista_deletar:
    df_hotmart.drop(i, inplace=True)

In [21]:
# ajustar valor do preço para int
for i in df_hotmart.index:
    valor = str(df_hotmart.at[i, 'preco'])
    valor = valor.replace('R$ ', '')
    valor = valor.replace(',', '.')
    df_hotmart.at[i, 'preco'] = valor

In [23]:
# ajustar qtd de avaliações
for i in df_hotmart.index:
    valor = str(df_hotmart.at[i, 'qtd_avaliação'])
    valor = valor.replace('(', '')
    valor = valor.replace(')', '')
    df_hotmart.at[i, 'qtd_avaliação'] = valor

In [29]:
# reenomear para formato do sheets
df_hotmart = df_hotmart[['nome_curso', 'autor', 'qtd_avaliação', 'avaliação', 'categoria', 'preco']]

In [31]:
# exportar para o excel
df_hotmart.to_excel('df_hotmart.xlsx', index=False)

In [32]:
# ler df
df_hotmart = pd.read_excel('df_hotmart.xlsx')

In [34]:
df_hotmart_desc = df_hotmart.copy()

In [35]:
df_hotmart = df_hotmart.drop(columns=['categoria'])

In [36]:
df_hotmart = df_hotmart.drop_duplicates()

In [42]:
df_hotmart['categoria'] = ''

In [44]:
for i in df_hotmart.index:
    nome = df_hotmart.at[i,'nome_curso']
    df_aux = df_hotmart_desc.loc[df_hotmart_desc['nome_curso']==nome]
    for aux in df_aux.index[:1]:
        valor = df_aux.at[aux,'categoria']
    df_hotmart.at[i,'categoria'] = valor


In [48]:
df_hotmart

,nome_curso,autor,qtd_avaliação,avaliação,preço,categoria
0,RESUMOS PARA VESTIBULARES & ENEM,Mariana Brentano,16.0,4.5,R$127.90,enem
3,"Ebook Histologia e Fisiologia - ENEM, vestibul...",Médica Leitora,11.0,4.8,R$50.00,enem
5,A Conquista da Física - Enem e Vestibulares,MARCOS SOARES RODRIGUES,7.0,5.0,R$197.00,enem
8,Flashcards Anki - ENEM e Vestibulares,Cláudia Rodrigues de Oliveira,5.0,5.0,R$50.00,enem
10,RESUMOS para VESTIBULARES e ENEM | PREMIUM @bi...,Amanda Brandt Beschorner,5.0,4.6,R$169.90,enem
...,...,...,...,...,...,...
110,Atalho Para Genialidade - ENEM e Vestibulares,julia rodrigues,NaN,NaN,R$147.00,pré-vestibular
111,RESUMOS para VESTIBULARES e ENEM | STANDARD H ...,Amanda Brandt Beschorner,NaN,NaN,R$99.90,pré-vestibular
119,Curso Preparatório Enem/Vestibular - Matemática,Vinicius Alves,NaN,NaN,R$29.90,vestibular
122,Aulas de redação nota 1000 para ENEM e vestibular,Bárbara Monteiro,NaN,NaN,R$200.00,vestibular


In [49]:
# reenomear para formato do sheets
df_hotmart = df_hotmart[['nome_curso', 'autor', 'qtd_avaliação', 'avaliação', 'categoria', 'preço']]

In [50]:
# exportar para o excel
df_hotmart.to_excel('df_hotmart.xlsx', index=False)